# TDoA-based Target Localization and Pursuit with Autonomous Vehicles: an Model Predictive Control Approach
The main aim of the work is to implement target pursuit with two surface trackers.

## Tracker Kinematic Model
We are going to use the simplest kinematic model of the tracker with only linear and angular velocities used as the input control.
$$
\begin{bmatrix}
\dot{x} \\
\dot{y} \\
\dot{\theta} \\
\end{bmatrix}
=
\begin{bmatrix}
v \cos{\theta} \\
v \sin{\theta} \\
\omega \\
\end{bmatrix}
=
\begin{bmatrix}
\cos{\theta} & 0 \\
\sin{\theta} & 0 \\
0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
v \\
\omega
\end{bmatrix}
$$
where $\mathbf{x} = [x, y, \theta]^\intercal$ are position and angle of the tracker in the inertial reference frame (the **state** vector) and $\mathbf{u} = [v, w]^\intercal$ are the input of he system (the **control** vector).

## Target Model
The tracker moves along the predefined trajectory $\mathbf{q}(t) = [q_x(t), q_y(t)]^\intercal \in \mathbb{R}^2$.
The discrete version of the tracker trajectory is
\begin{equation}
    \mathbf{q}_k = [q_{x,k}, q_{y,k}]
\end{equation}

In [ ]:
# Load CasADi library
from casadi import *
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# ---Define the discretization constants
DeltaT = 0.1 # Sampling time, c
PredictionN = 40 # Prediction horizon (prediction steps)

# ---Define the Control limitation
VLimit     = [0.0, 3.5]             # Limitation for vehicle linear velocity, m/s
OmegaLimit = [-np.pi / 2, np.pi / 2] # Limitation for vehicle angular velocity, rad/s

# Define Weight for running cost
Q = np.diag([1, 5, 0.1]) # Weight for the state vector, Matrix 3x3
R = np.diag([0.5, 0.05]) # Weight for the control vector, Matrix 2x2

In [ ]:
# ---Define Auxiliary variables
state_vector_size = 0 # State vector size
control_vector_size = 0 # Control vector size

# Define the states and state vector
x = SX.sym('x')
y = SX.sym('y')
theta = SX.sym('theta')
state_vector = vertcat(x, y, theta)
state_vector_size = state_vector.size()[0]

# Define the controls and control vector
v = SX.sym('v')
omega = SX.sym('omega')
control_vector = vertcat(v, omega)
control_vector_size = control_vector.size()[0]

# Define the Right-Hand Side (RHS) of kinematic model
rhs = np.array([[v * np.cos(theta)],
                [v * np.sin(theta)],
                [omega]])

# Define the symbolic kinematic model mapping of RHS for Casadi
# This mapping gives the prediction of the next state knowing current state and control
KinFunction = Function('KinFunction', [state_vector, control_vector], [rhs])

# Define symbolic parameter vector. It stores initial state (first n_states) and reference state (last n_states) the robot
SymInit = SX.sym('Init', state_vector_size + PredictionN * (state_vector_size + control_vector_size))

# Define the symbolic control prediction for all prediction steps
SymControl = SX.sym('Control', control_vector_size, PredictionN)

# Define the symbolic state predictions for all prediction steps + 1
SymPrediction = SX.sym('Prediction', state_vector_size, (PredictionN + 1))

# Define objective function
objective_fn = 0

for step in range(0, PredictionN):
    # Get state and control on the current step
    state = SymPrediction[:, step]
    control = SymControl[:, step]
    # Accumulate objective function for all steps in the prediction horizon
    period = state_vector_size + control_vector_size
    # Offset because first state_vector_size values reserved for initial state
    offset = state_vector_size
    # Target state to control on the current step of prediction
    target_state = SymInit[step * period + offset : step * period + offset + state_vector_size]
    # Change offset for the target control (this is initial state + first control state)
    offset = state_vector_size + state_vector_size
    # Target control
    target_control = SymInit[step * period + offset : step * period + offset + control_vector_size]
    objective_fn = objective_fn + \
        (state - target_state).T @ Q @ (state - target_state) + \
        (control - target_control).T @ R @ (control - target_control)

In [ ]:
# Temporary define equality constraints as a python list
constraints_equality = []
constraints_equality.append(SymPrediction[:, 0] - SymInit[0:3])

# Add constraints connected with kinematic model of the tracker 
for step in range(0, PredictionN):
    # Get the current and the next state
    state = SymPrediction[:, step]
    state_next = SymPrediction[:, step + 1]
    # Get the current control
    control = SymControl[:, step]
    # Compute rhs function value for the current state and control
    rhs_value = KinFunction(state, control)
    # Make prediction for the next step
    state_next_euler = state + (DeltaT * rhs_value)
    constraints_equality.append(state_next - state_next_euler)

ObstaclePosition = np.array([5.0, 15.0])
AvoidanceRadius = 1.0

# Add constraints connected with obstacle avoidance
# Just to check how it works
for step in range(0, PredictionN + 1):
    state = SymPrediction[:, step]
    constraints_equality.append(AvoidanceRadius - np.sqrt((state[0] - ObstaclePosition[0])**2 + (state[1] - ObstaclePosition[1])**2))

# Convert Python list to CasADi Matrix (vertical concatination)
constraints_equality = vertcat(*constraints_equality)

In [ ]:

# Reshape control variables (because we need to configure for NLP only one dimensional vector)
# But now we need to prepare the much larger vector of predictions of all states and controls within the receiving horizon
control_nlp = reshape(SymPrediction, state_vector_size * (PredictionN + 1), 1)
control_nlp = vertcat(control_nlp, reshape(SymControl, PredictionN * control_vector_size, 1))

# Configure the NLP solver (general view for all step within the prediction horizon)
nlp = {}
nlp['f'] = objective_fn         # objective function
nlp['x'] = control_nlp          # Control input as column vector
nlp['p'] = SymInit              # the initial state and the target state
nlp['g'] = constraints_equality # the initial state and the target state

# Define the configuration options for the solver
opts = {}
opts["print_time"] = 0
opts["verbose"] = False
opts["ipopt.max_iter"] = 100
opts["ipopt.print_level"] = 0
opts["ipopt.acceptable_tol"] = 1e-8
opts["ipopt.acceptable_obj_change_tol"] = 1e-6

# Initialize the solver
solver = nlpsol('solver', 'ipopt', nlp, opts)

# Initialize the global constraints (see constraints_equality)
# To get equality condition we need to set upper and lower bound parameters as zeros
args = {}
# Equality constraints by kinematic model 
args["lbg"] = np.zeros((state_vector_size * (PredictionN + 1) + PredictionN + 1, 1))
args["ubg"] = np.zeros((state_vector_size * (PredictionN + 1) + PredictionN + 1, 1))

# Inequality constraints by obstacle avoidance
args["lbg"][state_vector_size * (PredictionN + 1) : state_vector_size * (PredictionN + 1) + PredictionN + 1] = -inf
args["ubg"][state_vector_size * (PredictionN + 1) : state_vector_size * (PredictionN + 1) + PredictionN + 1] = 0

# Set the lower bound for the optimized vector (both state and control vector)
args["lbx"] = np.zeros((state_vector_size * (PredictionN + 1) + control_vector_size * PredictionN, 1))
# Fist let's fill the limit for state vector. First state_vector_size * (PredictionN + 1) part
# (x, y parts)
args["lbx"][0:(3 * PredictionN + 1) + 0:3] = -np.inf
args["lbx"][1:(3 * PredictionN + 1) + 1:3] = -np.inf
# (\phi part)
args["lbx"][2:(3 * PredictionN + 1) + 2:3] = -np.inf
# Second let's fill the limit for control vector. Second control_vector_size * PredictionN part
args["lbx"][state_vector_size * (PredictionN + 1) + 0:state_vector_size * (PredictionN + 1) + control_vector_size * PredictionN:2] = np.min(VLimit)
args["lbx"][state_vector_size * (PredictionN + 1) + 1:state_vector_size * (PredictionN + 1) + control_vector_size * PredictionN:2] = np.min(OmegaLimit)

# Set the upper bound for the optimized vector (both state and control vector) symmetricaly
args["ubx"] = np.zeros((control_vector_size * PredictionN + state_vector_size * (PredictionN + 1), 1))
args["ubx"][0:(3 * PredictionN + 1) + 0:3] = np.inf
args["ubx"][1:(3 * PredictionN + 1) + 1:3] = np.inf
args["ubx"][2:(3 * PredictionN + 1) + 2:3] = np.inf
args["ubx"][state_vector_size * (PredictionN + 1) + 0:state_vector_size * (PredictionN + 1) + control_vector_size * PredictionN:2] = np.max(VLimit)
args["ubx"][state_vector_size * (PredictionN + 1) + 1:state_vector_size * (PredictionN + 1) + control_vector_size * PredictionN:2] = np.max(OmegaLimit)

In [ ]:
from TrajectoryGenerator import TrajectoryGenerator

# Control points for the target trajectory 
control_points = np.array([[  5.0,  5.0],
                           [  2.0,  15.],
                           [ 10.0,  18.],
                           [ 15.,   15.],
                           [ 20.,   25.],
                           [ 15.,   30.]
                           ])

# Create trajectory instance
trajectory = TrajectoryGenerator(control_points)

# Get all list of normalized parameters to build the target trajectory
parameter_list = np.linspace(0, 1, 100)
trajectory_points = trajectory.get_trajectory_point(parameter_list)

# Display trajectory of the target
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The tracker trajectory on the plane')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_xlim(0, 25)
ax.set_ylim(0, 35)
ax.grid(True, linestyle="--", alpha=0.7)
ax.minorticks_on()

ax.plot(trajectory_points[:, 0], trajectory_points[:, 1], '--', label='Target Trajectory', color='c', zorder=2)
ax.plot(control_points[:, 0], control_points[:, 1], '-o', label='Control Points', color='b', zorder=2)
ax.scatter(trajectory_points[0, 0], trajectory_points[0, 1], marker='o', s=50, label='Start', color='g', zorder=3)
ax.scatter(trajectory_points[-1, 0], trajectory_points[-1, 1], marker='o', s=50, label='Finish', color='r', zorder=3)
ax.legend(loc='upper left')

In [ ]:
# Configuration for the simulation loop
SimulationTime = 25   # Final simulation time
ErrTolerance = 2.0e-2 # Final position tolerance

# The initial state of the tracker
StateInitial = np.array([[2.0], [2.0], [0]]) # Initial position

# Velocity of the target
TargetVelocity = 2.0

# Init the first control inputs.
# We don't have any prediction, so zeros are our best guess
control_guess = np.zeros((PredictionN, control_vector_size))
# The state on the first point will be not far of the StateInitial
state_guess = repmat(StateInitial, 1, PredictionN + 1)

# Storage for the states throughout the simulation (python list of vectors)
storage_states = []

# Storage for predictions (python list of 2D matrix) throughout the simulation
storage_predictions = []

# Storage for control actions (2D matrix: (cumulative) x Control Vector Size)
storage_control = []

# Storage for target states
storage_target_states = []

# Current state of the robot. At the beginning it coincides with the x_start
state_current = StateInitial

# Start timer to check performance
time_start = time.process_time()

# Current time of simulation
time_sim = 0

# Trajectory parameter
parameter = 0

# Main Loop of the motion control
while parameter < 1.0:

    # Get target coordinate
    target_point = trajectory.get_trajectory_point(parameter)

    # Calculate tangent vector at the point
    trajectory_derivative = trajectory.get_trajectory_derivative(parameter)
    target_yaw = np.arctan2(trajectory_derivative[1], trajectory_derivative[0])

    # Configure target state
    target_state = np.append(target_point, target_yaw).reshape((-1, 1))

    # Upload new initial state (new each step) and the target state (for each point within the prediction horizon)
    # First part is current state
    initial_vector = []
    initial_vector.append(state_current.flatten())
    parameter_prediction = parameter
    # After we iteratively append target state and target control
    for st in range(0, PredictionN):
        # check is this is the end of trajectory
        terminal_step = parameter_prediction > 1.0

        # Limit the trajectory parameter for predictions
        if terminal_step:
            parameter_prediction = 1.0

        # Get target coordinate
        target_point_pred = trajectory.get_trajectory_point(parameter_prediction)

        # Calculate tangent vector at the point
        trajectory_derivative_pred = trajectory.get_trajectory_derivative(parameter_prediction)
        target_yaw_pred = np.arctan2(trajectory_derivative_pred[1], trajectory_derivative_pred[0])

        # Configure target state
        target_state_prediction = np.append(target_point_pred, target_yaw_pred)

        # Calculate a new parameter delta to preserve velocity constant
        parameter_delta_delta = TargetVelocity * DeltaT / np.linalg.norm(trajectory_derivative_pred)
        parameter_prediction = parameter_prediction + parameter_delta_delta

        # Add state vector within the horizon
        initial_vector.append(target_state_prediction)
        # Add control vector within the horizon (zero linear velocity if this is the terminal state)
        initial_vector.append(np.array([TargetVelocity if not terminal_step else 0.0, 0.0]))

    # Reshape the initial vector to standard representation
    initial_vector = np.concatenate(initial_vector, axis=0)
    initial_vector = initial_vector.reshape(-1, 1)

    args["p"]  = initial_vector

    # Set initial guess
    args["x0"] = reshape(state_guess, state_vector_size * (PredictionN + 1), 1)
    args["x0"] = vertcat(args["x0"], reshape(control_guess.T, control_vector_size * PredictionN, 1))

    # Store the state vector on the each step of simulation
    storage_states.append(state_current)

    # Solve the OCP
    sol = solver(x0=args["x0"], p=args["p"],
                 lbx=args["lbx"], ubx=args["ubx"],
                 lbg=args["lbg"], ubg=args["ubg"]
                 )

    # Reshape obtained contol predictions vector to matrix (2xPredictionN)
    # Extract only control predictions
    control_predictions = reshape(sol['x'][state_vector_size * (PredictionN + 1):].T, control_vector_size, PredictionN).T
    # We need only the first control value in the storage
    control = control_predictions[0, :]

    # Extract only state predictions
    state_predictions = reshape(sol['x'][:state_vector_size * (PredictionN + 1)].T, state_vector_size, PredictionN + 1)

    # Store predicted states in the list of 2D Matrix
    storage_predictions.append(state_predictions.T)

    # Store only the first control value in the storage
    storage_control.append(control)
    
    # Store current target state
    storage_target_states.append(target_state)

    # Shift the time and initial position for solver
    # We propagate control actions to obtain the new state of the system at this point
    state_current = state_current + (DeltaT *  KinFunction(state_current, control.T))
    state_current = np.array(state_current)
    time_sim = time_sim + DeltaT

    # Calculate a new parameter delta to preserve velocity constant
    parameter_delta = TargetVelocity * DeltaT / np.linalg.norm(trajectory_derivative)
    parameter = parameter + parameter_delta

    # We the control action predictions and state predictions as the best guess for the next control
    # Just trim the first row, because we already used it and append zero row at the end
    control_guess = np.delete(control_predictions, 0, axis=0)
    control_guess = np.vstack((control_guess, control_predictions[-1, :]))

    state_guess = np.delete(state_predictions, 0, axis=1)
    state_guess = np.hstack((state_guess, state_predictions[:, -1]))

# Simulation steps number
simulation_steps = len(storage_control)

elapsed_time = time.process_time() - time_start
print(f"Overal OCP time for all steps: {elapsed_time} sec. Per step: {elapsed_time / simulation_steps}")

# Convert list of matrix to 3D np matrix and put list into the 3rd dimension
storage_predictions = np.stack(storage_predictions, axis=2)

# Concatinate all data along the second axis (because state is the column-vector)
storage_states = np.concatenate(storage_states, axis=1)

# Concatinate all data along the second axis (because state is the column-vector)
storage_target_states = np.concatenate(storage_target_states, axis=1)

# Concatinate all data along the first axis (because control is the row-vector)
storage_control = np.concatenate(storage_control, axis=0)

In [ ]:
from IPython import display
import utility as ut
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The vehicle movement on the plane')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_xlim(0, 25)
ax.set_ylim(0, 35)
ax.grid(True, linestyle="--", alpha=0.7)

# Heading Figure size
markersize = 40

# Plot the target trajectory
ax.plot(trajectory_points[:, 0], trajectory_points[:, 1], '--', label='Target Trajectory', color='c', zorder=2)

for i in range(simulation_steps):
    marker_tracker, scale = ut.gen_arrow_head_marker(storage_states[2, i])
    pointer_tracker = ax.scatter(storage_states[0, i], storage_states[1, i],
                                 marker=marker_tracker, s=(markersize*scale)**2, color='green', zorder=3)
    
    marker_target, scale = ut.gen_arrow_head_marker(storage_target_states[2, i])
    pointer_target = ax.scatter(storage_target_states[0, i], storage_target_states[1, i],
                                marker=marker_target, s=(markersize*scale)**2, color='red', zorder=3)
    
    trajectory_actual = ax.plot(storage_states[0, 0:i], storage_states[1, 0:i], 
                                linestyle="--", linewidth=1.5, color='black', 
                                label='Tracker Trajectory', zorder=1)[0]
    trajectory_predict = ax.scatter(storage_predictions[:, 0, i], storage_predictions[:, 1, i],
                                    color='purple', marker='x', s=10, label='Tracker Trajectory Predicted', zorder=1)
    ax.legend(loc='upper left')

    # Update plot rendering
    display.clear_output(wait=True)
    display.display(fig)

    # Wait to render the current step of the trajectory
    # with 5x faster than simulation process
    fast_forward_scale = 5
    time.sleep(DeltaT / fast_forward_scale)

    # Remove pointer in order to redraw it on the next stage
    trajectory_predict.remove()
    trajectory_actual.remove()
    pointer_tracker.remove()
    pointer_target.remove()

plt.close(fig)